In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Helper libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
from timeit import default_timer as timer

# Custom libraries
from tensorflow_generator import TensorflowDataGenerator, TensorflowDataGenerator_Test

In [18]:
#### Set the CNN parameters
num_epoch = 5
batch_size = 6
framework = "tensorflow"
train_type = "manual"
im_size = 224
num_im = 1000
predict = True
model_type="simple"

In [3]:
# Set up directories
root_path = "../"
data_dir = os.path.join(root_path, "dogs-vs-cats/train")
log_dir = os.path.join(
    root_path,
    "logs",
    framework,
    model_type,
    "logs",
    train_type + "_" + datetime.now().strftime("%Y%m%d-%H%M%S"),
)
if predict:
    ckpt_name = f"{train_type}_20200816-184346"
    ckpt_dir = os.path.join(
        root_path, "logs", framework, model_type, "ckpts", ckpt_name
    )
else:
    ckpt_dir = os.path.join(
        root_path,
        "logs",
        framework,
        model_type,
        "ckpts",
        train_type + "_" + datetime.now().strftime("%Y%m%d-%H%M%S"),
    )

In [4]:
# Setup input pipeline
train_dir = os.path.join(root_path, "dogs-vs-cats/train")
test_dir = os.path.join(root_path, "dogs-vs-cats/test")

train_gen = TensorflowDataGenerator(
    train_dir, batch_size, im_size=im_size, num_im=num_im, shuffle=True
)
val_imgs = train_gen.load_val()
test_gen = TensorflowDataGenerator_Test(test_dir, batch_size, im_size=im_size)

In [17]:
# Set up model option 1
model = Sequential(
    [
        layers.Conv2D(
            32,
            3,
            padding="same",
            activation="relu",
            input_shape=(im_size, im_size, 3),
        ),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(1),
    ]
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 28, 28, 128)       1

In [10]:
# Set up model option 1.2
model = Sequential()
model.add(layers.Conv2D(
        32,
        3,
        padding="same",
        activation="relu",
        input_shape=(im_size, im_size, 3)
    ))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, 3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, 3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1))

In [30]:
# Set up model option 2
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(im_size, im_size, 3),)
x = layers.Conv2D(32, 3, padding="same", activation="relu")(input_layer)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
output_layer = layers.Dense(1)(x)

model = Model(inputs=input_layer, outputs=[output_layer])

In [31]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 28, 28, 128)       0   

In [39]:
# Set up model option 3
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = layers.Conv2D(32, 3, padding="same", activation="relu")
        self.pool = layers.MaxPooling2D()
        self.conv2 = layers.Conv2D(64, 3, padding="same", activation="relu")
        self.conv3 = layers.Conv2D(128, 3, padding="same", activation="relu")
        self.conv4 = layers.Conv2D(128, 3, padding="same", activation="relu")
        self.flatten = layers.Flatten()
        self.dropout = layers.Dropout(0.5)
        self.fc1 = layers.Dense(512, activation="relu")
        self.fc2 = layers.Dense(1)
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
model = CustomModel()

In [6]:
# Compile model
learning_rate = 1e-3

# Get model summary
# model.summary()

In [42]:
def my_huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [48]:
if train_type == "auto":
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

    # Setup callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_dir,
        save_weights_only=True,
        monitor="val_acc",
        mode="max",
        save_best_only=True,
    )

    model.compile(loss=my_huber_loss, optimizer=optimizer, metrics=["accuracy"])

    # train the model
    history = model.fit(
        train_gen,
        validation_data=val_imgs,
        validation_steps=len(val_imgs[0]) // batch_size,
        epochs=num_epoch,
        callbacks=[tensorboard_callback, model_checkpoint_callback],
        use_multiprocessing=True,
        workers=8,
    )

elif train_type == "manual":
    # loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss_fn = my_huber_loss
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

    train_acc = tf.keras.metrics.Mean()
    train_loss = tf.keras.metrics.Mean()
    val_acc = tf.keras.metrics.Mean()
    val_loss = tf.keras.metrics.Mean()

    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            logits = model(x, training=True)
            loss_value = loss_fn(y, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        acc_value = tf.math.equal(
            y, tf.math.round(tf.keras.activations.sigmoid(logits))
        )
        train_acc.update_state(acc_value)
        train_loss.update_state(loss_value)

    @tf.function
    def test_step(x, y):
        val_logits = model(x, training=False)
        loss_value = loss_fn(y, val_logits)
        acc_value = tf.math.equal(
            y, tf.math.round(tf.keras.activations.sigmoid(val_logits))
        )
        val_acc.update_state(acc_value)
        val_loss.update_state(loss_value)
        return loss_value, acc_value

    # Setup tensorboard
    file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
    best_val_acc = 0  # for model check pointing
    # Epoch loop
    for epoch in range(1, num_epoch + 1):
        start_time = timer()
        # Training loop
        for inputs, targets in train_gen:
            train_step(inputs, targets)

        # Validation loop
        for batch_idx in range(0, len(val_imgs[1]), batch_size):
            inputs = val_imgs[0][batch_idx : batch_idx + batch_size, ...]
            targets = val_imgs[1][batch_idx : batch_idx + batch_size]
            test_step(inputs, targets)

        # Log metrics to tensorboard
        end_time = timer()
        with file_writer.as_default():
            tf.summary.scalar("Loss/train", train_loss.result(), step=epoch)
            tf.summary.scalar("Loss/validation", val_loss.result(), step=epoch)
            tf.summary.scalar("Accuracy/train", train_acc.result(), step=epoch)
            tf.summary.scalar("Accuracy/validation", val_acc.result(), step=epoch)
            tf.summary.scalar("epoch_time", end_time - start_time, step=epoch)

        # Display metrics at the end of each epoch.
        print(
            f"Epoch: {epoch} \tTraining Loss: {train_loss.result()} \tValidation Loss: {val_loss.result()} \tTraining Accuracy: {train_acc.result()} \tValidation Accuracy: {val_acc.result()} \tTime taken: {end_time - start_time}"
        )

        # checkpoint if improved
        if val_acc.result() > best_val_acc:
            model.save_weights(ckpt_dir)
            best_val_acc = val_acc.result()

        # Reset training metrics at the end of each epoch
        train_acc.reset_states()
        train_loss.reset_states()
        val_acc.reset_states()
        val_loss.reset_states()

/home/kokubuntu/.pyenv/versions/3.9.10/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch: 1 	Training Loss: 0.0005148930940777063 	Validation Loss: 0.00018744784756563604 	Training Accuracy: 1.0 	Validation Accuracy: 1.0 	Time taken: 4.099570872000186
Epoch: 2 	Training Loss: 0.0002657287986949086 	Validation Loss: 0.00011425477714510635 	Training Accuracy: 1.0 	Validation Accuracy: 1.0 	Time taken: 3.531686952999735
Epoch: 3 	Training Loss: 0.0003042563912458718 	Validation Loss: 0.00010934464808087796 	Training Accuracy: 1.0 	Validation Accuracy: 1.0 	Time taken: 3.5307279200005723
Epoch: 4 	Training Loss: 0.00028295995434746146 	Validation Loss: 3.576412927941419e-05 	Training Accuracy: 1.0 	Validation Accuracy: 1.0 	Time taken: 3.780366449000212
Epoch: 5 	Training Loss: 0.0004524594114627689 	Validation Loss: 0.00036074992385692894 	Training Accuracy: 1.0 	Validation Accuracy: 1.0 	Time taken: 3.6673804660003952


In [ ]:
model.load_weights(ckpt_dir)
for inputs in test_gen:
    outputs = model(inputs, training=False)
    break
print(outputs)